In [ ]:
%load_ext autoreload
%autoreload 2
from ml.ingest.textocr_to_torch import TextOCRDoctrDetDataset, doctr_detection_collate
from torch.utils.data import DataLoader
from doctr.models import db_resnet50, DBNet
from pathlib import Path
import torch

In [ ]:
LR = 0.001
BATCH_SIZE = 4
MODEL_DIR = Path('../../model')

dataset = TextOCRDoctrDetDataset(num_samples=32)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=doctr_detection_collate)

In [ ]:
model: DBNet = db_resnet50(pretrained=False).train()

optimizer = torch.optim.Adam([p for p in model.parameters() if p.requires_grad], LR)

for batch, (images, targets) in enumerate(loader):
    optimizer.zero_grad(set_to_none=True)
    train_loss = model(images, targets)["loss"]
    train_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
    optimizer.step()

    print(batch, train_loss.item())

    if batch >= 5:
        break

In [ ]:
torch.save(model.state_dict(), MODEL_DIR / 'dbnet_textocr.pt')